In [1]:
import numpy as np
import astropy
import statistics
from scipy import stats as scistats
from scipy.stats import wilcoxon
from scipy.stats import binomtest
from scipy.stats import f_oneway
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp
from scipy.stats import shapiro

from astropy import stats
from statistics import multimode

In [2]:
correct = np.load('correct.npy') #Counts of correct answers by question
experts_correct = np.load('experts_correct.npy')
nonexperts_correct = np.load('nonexperts_correct.npy')
astromus_correct = np.load('astromus_correct.npy')
mus_correct = np.load('mus_correct.npy')
astro_correct = np.load('astro_correct.npy')
nonexperts_4_correct = np.load('nonexperts_4_correct.npy')
blv_correct = np.load('blv_correct.npy')
nonblv_correct = np.load('nonblv_correct.npy')

In [3]:
grades = np.load('grades.npy') #Counts of correct answers by participant
grades_experts = np.load('grades_experts.npy')
grades_nonexperts = np.load('grades_nonexperts.npy')
grades_astromus = np.load('grades_astromus.npy')
grades_mus = np.load('grades_mus.npy')
grades_astro = np.load('grades_astro.npy')
grades_nonexperts_4 = np.load('grades_nonexperts_4.npy')
grades_nonblv = np.load('grades_nonblv.npy')
grades_nonblv = np.load('grades_nonblv.npy')

In [4]:
groups = ["Global", "Experienced", "Non-experienced", "Astro", "Mus", "Astromus", "Nothing","BLV", "Non-BLV-2"]
counts = [correct, experts_correct, nonexperts_correct, astro_correct, mus_correct, astromus_correct, nonexperts_4_correct]
participants = [38, 16, 16, 3, 4, 4, 4, 2, 2]

In [5]:
questions = len(correct)
questions

4

In [6]:
null_value = 0.14

# Statistics

In [7]:
#mean, median, mode
def calculations(counts, participants, questions):
    print("Participants:", participants)
    print(counts)
    mean = np.round(np.mean(counts/participants), 4) #overall average success rate
    median = np.round(np.median(counts/participants), 4) #median 
    mode = multimode(np.round(counts, 2))
    return mean, median, mode

In [8]:
null_value

0.14

In [9]:
# Wilcoxon test. Calculates if the observed overall average success rate
#is statistically significantly higher than chance.

def wilcoxon_test(counts, participants, null_value):
    observed_proportions = [x/participants for x in counts]
    # Compute differences from the null value
    differences = [x - null_value for x in observed_proportions]
    # Perform the Wilcoxon signed-rank test
    # alternative='greater' tests whether the median is significantly greater than the null
    test_statistic, p_value = wilcoxon(differences, alternative='greater')
    # Output results
    print("Success rates:", observed_proportions)
    print("Test statistic (W):", test_statistic)
    print("p-value (one-sided):", p_value)
    if p_value <= 0.05:
        print("Conclusion: Statistically significant")
    else:
        print("Conclusion: Not statistically significant")
    return test_statistic, p_value

In [10]:
# Alternative Binomial test
def binom_test(counts, participants, questions, null_value, less_or_greater):
    questions = questions-2
    test = binomtest(np.sum(counts), participants*questions, p=null_value, alternative=less_or_greater)
    if test.pvalue<0.05:
        print("Statistically significant")
    else:
        print("Not statistically significant")
    return test

In [11]:
# Alternative T-test
def t_test(counts, questions, null_value):  # Perform a one-sample t-test
    sample_mean = np.mean(counts)
    n = len(counts)
    sample_std = np.std(counts, ddof=1)  # Use sample standard deviation (ddof=1 for unbiased estimate)
    standard_error = sample_std / np.sqrt(n)
    test_statistic = (sample_mean - null_value) / standard_error
    degrees_freedom = n - 1
    p_value = scistats.t.sf(np.abs(test_statistic), df=degrees_freedom) * 2  # Two-tailed p-value
    if p_value <= 0.05:
        result = "Reject the null hypothesis. There is statistically significant evidence that the mean is different from the random choice value."
    else:
        result = "Fail to reject the null hypothesis. There is not enough statistically significant evidence that the mean is different from the random choice value."

    return test_statistic, p_value

# 1- Are the overall average success rates observed significant vs chance?

In [12]:
calculations(correct, participants[0], questions)

Participants: 38
[14 15 16 17]


(0.4079, 0.4079, [14, 15, 16, 17])

In [13]:
t_test(correct, questions, null_value)

(23.79560967909837, 0.00016263946590045782)

### Conclusion: All the overall average success rates observed with the exception of the astronomers group are significantly higher than the random choice probability

In [14]:
calculations(experts_correct, participants[1], questions)

Participants: 16
[ 7  8  6 10]


(0.4844, 0.4688, [7, 8, 6, 10])

In [15]:
t_test(experts_correct, questions, null_value)

(8.911919466166006, 0.002979982946520602)

In [16]:
calculations(nonexperts_correct, participants[2], questions)

Participants: 16
[6 5 8 6]


(0.3906, 0.375, [6])

In [17]:
t_test(nonexperts_correct, questions, null_value)

(9.711471242000902, 0.0023188989956530043)

In [18]:
calculations(astro_correct, participants[3], questions)

Participants: 3
[0 0 1 0]


(0.0833, 0.0, [0])

In [19]:
t_test(astro_correct, questions, null_value)

(0.43999999999999995, 0.6897076291875928)

In [20]:
calculations(mus_correct, participants[4], questions)

Participants: 4
[3 2 2 4]


(0.6875, 0.625, [2])

In [21]:
t_test(mus_correct, questions, null_value)

(5.4521121845324565, 0.012120763019798014)

In [22]:
calculations(astromus_correct, participants[5], questions)

Participants: 4
[2 3 1 2]


(0.5, 0.5, [2])

In [23]:
t_test(astromus_correct, questions, null_value)

(4.556050921576711, 0.019819032056035017)

In [24]:
calculations(nonexperts_4_correct, participants[6], questions)

Participants: 4
[2 2 2 1]


(0.4375, 0.5, [2])

In [25]:
t_test(nonexperts_4_correct, questions, null_value)

(6.4399999999999995, 0.007591782352051938)

# 2- Are the differences observed in the means statistically significant?

In [51]:
f_stat, p_valor = scistats.f_oneway(experts_correct, nonexperts_correct)

print("Estadístico F:", f_stat)
print("Valor p:", p_valor)

Estadístico F: 2.0
Valor p: 0.20703125


In [26]:
# Kruskal-Wallis test: kruskal_p_value < 0.05 => reject the null hypothesis => statistical significance
H_statistic, kruskal_p_value = scistats.kruskal(experts_correct, nonexperts_correct)
print(H_statistic, kruskal_p_value)

1.7943037974683544 0.18040266628975238


### Conclusion: The differences in the means between experts and non experts are not statistically significant.

In [45]:
# T-test
stat, p_value = ttest_ind(experts_correct, nonexperts_correct, equal_var=False)
print(stat, p_value)
deg_free = len(experts_correct) - 1
print(deg_free)

1.4142135623730951 0.21118278615730318
3


# 3- Are they meaningful?

In [28]:
# Cohen's d for Effect Size (T-test). Meaninful: Small effect size d=0.2, medium d=0.5, large d=0.8.

# Calculate means and standard deviations
mean1 = np.mean(experts_correct/participants[1])
mean2 = np.mean(nonexperts_correct/participants[2])
std1 = np.std(experts_correct, ddof=1)
std2 = np.std(nonexperts_correct, ddof=1)

# Calculate Cohen's d
cohen_d = (mean1 - mean2) / np.sqrt((std1**2 + std2**2) / 2)
print("Cohen's d:", cohen_d)

Cohen's d: 0.0625


# 4- Subgroup analysis

In [29]:
# Test normality for each group. p-value<0.05 =>not normally distributed
stat1, p1 = shapiro(astromus_correct)
stat2, p2 = shapiro(mus_correct)
stat3, p3 = shapiro(astro_correct)
stat4, p4 = shapiro(nonexperts_4_correct)

print(f"astromus normality: p={p1}")
print(f"mus normality: p={p2}")
print(f"astro normality: p={p3}")
print(f"nonexperts normality: p={p4}")

astromus normality: p=0.6829614043235779
mus normality: p=0.27245327830314636
astro normality: p=0.001240724348463118
nonexperts normality: p=0.001240724348463118


In [30]:
# Kruskal-Wallis test: kruskal_p_value < 0.05 => reject the null hypothesis => statistical significance
H_statistic, kruskal_p_value = scistats.kruskal(astromus_correct, mus_correct, astro_correct, nonexperts_4_correct)
print(H_statistic, kruskal_p_value)

10.15243902439025 0.017313816405040983


In [31]:
#alternative test: Chi square
stat, p_value = friedmanchisquare(astromus_correct, mus_correct, astro_correct, nonexperts_4_correct)
print(stat, p_value)

7.916666666666666 0.04776571858126227


### Post-hoc analysis

In [49]:
# Dunn test without Bonferroni correction (6 comparisons => alpha = 0.05/6 = 0.0083)
dunn_result = sp.posthoc_dunn([astromus_correct, mus_correct, astro_correct, nonexperts_4_correct])#, p_adjust='bonferroni')

# Label the groups
dunn_result.index = ['astromus', 'mus', 'astro', 'nothing']
dunn_result.columns = ['astromus', 'mus', 'astro', 'nothing']

print(dunn_result)

          astromus       mus     astro   nothing
astromus  1.000000  0.390365  0.026049  0.725295
mus       0.390365  1.000000  0.002039  0.226146
astro     0.026049  0.002039  1.000000  0.060919
nothing   0.725295  0.226146  0.060919  1.000000


In [50]:
# Dunn test with Bonferroni correction (6 comparisons => alpha = 0.05/6 = 0.0083)
dunn_result = sp.posthoc_dunn([astromus_correct, mus_correct, astro_correct, nonexperts_4_correct], p_adjust='bonferroni')

# Label the groups
dunn_result.index = ['astromus', 'mus', 'astro', 'nothing']
dunn_result.columns = ['astromus', 'mus', 'astro', 'nothing']

print(dunn_result)

          astromus       mus     astro   nothing
astromus  1.000000  1.000000  0.156296  1.000000
mus       1.000000  1.000000  0.012236  1.000000
astro     0.156296  0.012236  1.000000  0.365512
nothing   1.000000  1.000000  0.365512  1.000000


## Conclusion: astro group is significantly different from the rest of the groups, with large (0,82 - mus) and medium effect size (~0.5 - astromus and nonexperts). With Bonferroni correction the differences are statistically significant only with the mus group.

In [33]:
# T-test Check
t_stat, p_value = ttest_ind(mus_correct, astro_correct)
print(t_stat, p_value)

4.629100498862757 0.0035810662448476607


In [34]:
t_stat, p_value = ttest_ind(astromus_correct, astro_correct)
print(t_stat, p_value)

3.6556307750696546 0.010634679730855207


In [35]:
t_stat, p_value = ttest_ind(nonexperts_4_correct, astro_correct)
print(t_stat, p_value)

4.242640687119285 0.005423950341308747


In [36]:
# Cohen's d for Effect Size (T-test). Meaninful: Small effect size d=0.2, medium d=0.5, large d=0.8.

# Calculate means and standard deviations
mean1 = np.mean(mus_correct/participants[4])
mean2 = np.mean(astro_correct/participants[5])
std1 = np.std(mus_correct, ddof=1)
std2 = np.std(astro_correct, ddof=1)

# Calculate Cohen's d
cohen_d = (mean1 - mean2) / np.sqrt((std1**2 + std2**2) / 2)
print("Cohen's d:", cohen_d)

Cohen's d: 0.8183170883849715


In [37]:
# Cohen's d for Effect Size (T-test). Meaninful: Small effect size d=0.2, medium d=0.5, large d=0.8.

# Calculate means and standard deviations
mean1 = np.mean(astromus_correct/participants[4])
mean2 = np.mean(astro_correct/participants[5])
std1 = np.std(mus_correct, ddof=1)
std2 = np.std(astro_correct, ddof=1)

# Calculate Cohen's d
cohen_d = (mean1 - mean2) / np.sqrt((std1**2 + std2**2) / 2)
print("Cohen's d:", cohen_d)

Cohen's d: 0.5728219618694801


In [38]:
# Cohen's d for Effect Size (T-test). Meaninful: Small effect size d=0.2, medium d=0.5, large d=0.8.

# Calculate means and standard deviations
mean1 = np.mean(nonexperts_4_correct/participants[4])
mean2 = np.mean(astro_correct/participants[5])
std1 = np.std(mus_correct, ddof=1)
std2 = np.std(astro_correct, ddof=1)

# Calculate Cohen's d
cohen_d = (mean1 - mean2) / np.sqrt((std1**2 + std2**2) / 2)
print("Cohen's d:", cohen_d)

Cohen's d: 0.4909902530309829


# BLV Analysis

In [39]:
calculations(blv_correct, participants[7], questions)

Participants: 2
[0 0 1 1]


(0.25, 0.25, [0, 1])

In [40]:
t_test(blv_correct, questions, null_value)

(1.2470765814495917, 0.300860089154109)

### Conclusion: The BLV average success rate difference with the random choice probability is not statistically significant.

In [41]:
calculations(nonblv_correct, participants[8], questions)

Participants: 2
[0 1 2 1]


(0.5, 0.5, [1])

In [42]:
t_test(nonblv_correct, questions, null_value)

(2.106561178793533, 0.12577756994323236)

In [43]:
# Kruskal-Wallis test: kruskal_p_value < 0.05 => reject the null hypothesis => statistical significance
H_statistic, kruskal_p_value = scistats.kruskal(blv_correct, nonblv_correct)
print(H_statistic, kruskal_p_value)

0.8999999999999999 0.34278171114790873


### Conclusion: The differences between BLV and non BLV average success rates are not statistically significant.